In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv("/home/jvdboss/workspace/ML_DL/backorder/artifacts/raw_local_dir/data.csv")
pd.set_option('display.max_rows', False)
pd.set_option('display.max_columns', False)

/home/jvdboss/anaconda3/envs/backorder/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,3285085,62.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,No,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
1,3285131,9.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,No,0.0,-99.00,-99.00,0.0,No,No,Yes,No,No,No
2,3285358,17.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,0.92,0.95,0.0,No,No,No,Yes,No,No
3,3285517,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,No,0.0,0.78,0.75,0.0,No,No,Yes,Yes,No,No
4,3285608,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,0.54,0.71,0.0,No,No,No,Yes,No,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sku                242076 non-null  object 
 1   national_inv       242075 non-null  float64
 2   lead_time          227351 non-null  float64
 3   in_transit_qty     242075 non-null  float64
 4   forecast_3_month   242075 non-null  float64
 5   forecast_6_month   242075 non-null  float64
 6   forecast_9_month   242075 non-null  float64
 7   sales_1_month      242075 non-null  float64
 8   sales_3_month      242075 non-null  float64
 9   sales_6_month      242075 non-null  float64
 10  sales_9_month      242075 non-null  float64
 11  min_bank           242075 non-null  float64
 12  potential_issue    242075 non-null  object 
 13  pieces_past_due    242075 non-null  float64
 14  perf_6_month_avg   242075 non-null  float64
 15  perf_12_month_avg  242075 non-null  float64
 16  lo

In [5]:
df.describe()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
count,2.420750e+05,227351.000000,242075.000000,2.420750e+05,2.420750e+05,2.420750e+05,242075.000000,2.420750e+05,2.420750e+05,2.420750e+05,242075.000000,242075.000000,242075.000000,242075.000000,242075.000000
mean,4.997510e+02,7.923018,36.178213,1.814723e+02,3.488073e+02,5.082963e+02,51.478195,1.721393e+02,3.404254e+02,5.117754e+02,52.804693,1.824236,-7.093779,-6.632445,0.843726
std,2.928039e+04,7.041410,898.673127,5.648875e+03,1.008180e+04,1.410972e+04,1544.678350,5.164244e+03,9.386523e+03,1.397670e+04,1278.591177,178.679263,26.900636,26.160720,45.606626
min,-2.541400e+04,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,-99.000000,-99.000000,0.000000
25%,4.000000e+00,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.630000,0.660000,0.000000
50%,1.500000e+01,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000e+00,2.000000e+00,4.000000e+00,0.000000,0.000000,0.820000,0.810000,0.000000
75%,8.100000e+01,9.000000,0.000000,4.000000e+00,1.200000e+01,2.000000e+01,4.000000,1.400000e+01,3.000000e+01,4.600000e+01,3.000000,0.000000,0.960000,0.950000,0.000000
max,1.214579e+07,52.000000,265272.000000,1.510592e+06,2.157024e+06,3.162260e+06,349620.000000,1.099852e+06,2.103389e+06,3.195211e+06,303713.000000,79964.000000,1.000000,1.000000,6232.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sku                242076 non-null  object 
 1   national_inv       242075 non-null  float64
 2   lead_time          227351 non-null  float64
 3   in_transit_qty     242075 non-null  float64
 4   forecast_3_month   242075 non-null  float64
 5   forecast_6_month   242075 non-null  float64
 6   forecast_9_month   242075 non-null  float64
 7   sales_1_month      242075 non-null  float64
 8   sales_3_month      242075 non-null  float64
 9   sales_6_month      242075 non-null  float64
 10  sales_9_month      242075 non-null  float64
 11  min_bank           242075 non-null  float64
 12  potential_issue    242075 non-null  object 
 13  pieces_past_due    242075 non-null  float64
 14  perf_6_month_avg   242075 non-null  float64
 15  perf_12_month_avg  242075 non-null  float64
 16  lo

In [7]:
df.isnull().sum()

sku                      0
national_inv             1
lead_time            14725
in_transit_qty           1
forecast_3_month         1
forecast_6_month         1
forecast_9_month         1
sales_1_month            1
sales_3_month            1
sales_6_month            1
sales_9_month            1
min_bank                 1
potential_issue          1
pieces_past_due          1
perf_6_month_avg         1
perf_12_month_avg        1
local_bo_qty             1
deck_risk                1
oe_constraint            1
ppap_risk                1
stop_auto_buy            1
rev_stop                 1
went_on_backorder        1
dtype: int64

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
def process(df):
    """
    Some strategies adopted:
    - Binaries were converted from strings ('Yes' and 'No') to 1 and 0.
    - The attributes related to quantities were normalized (std dev equal to 1)
    per row. Therefore, parts with different order of magnitudes are 
    approximated. For example: 1 unit of a expensive machine may be different 
    from 1 unit of a screw, but if we standard deviate all the quantities we 
    have, we can get a better proportion of equivalence between those items.
    - Missing values for lead_time and perf_month_avg were replaced using 
    series median and mean. 
    """
    # Imput missing lines and drop line with problem

    df['lead_time'] = SimpleImputer(strategy='median').fit_transform(
                                    df['lead_time'].values.reshape(-1, 1))
    df = df.dropna()
    for col in ['perf_6_month_avg', 'perf_12_month_avg']:
        df[col] = SimpleImputer(missing_values=-99).fit_transform(
                                    df[col].values.reshape(-1, 1))
    # Convert to binaries
    for col in ['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk',
               'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
        df[col] = (df[col] == 'Yes').astype(int)
    # Normalization    
    from sklearn.preprocessing import normalize
    qty_related = ['national_inv', 'in_transit_qty', 'forecast_3_month', 
                   'forecast_6_month', 'forecast_9_month', 'min_bank',
                   'local_bo_qty', 'pieces_past_due', 'sales_1_month', 
                   'sales_3_month', 'sales_6_month', 'sales_9_month',]
    df[qty_related] = normalize(df[qty_related], axis=1)
    df = df.astype(float)   
    return df


In [26]:
new_df=process(df)
new_df

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,3285085.0,0.999870,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016127,0.0,0.0,0.77934,0.776331,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,3285131.0,0.993884,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110432,0.0,0.0,0.77934,0.776331,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3285358.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.92000,0.950000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3285517.0,0.976187,2.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.216930,0.000000,0.0,0.0,0.78000,0.750000,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,3285608.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.54000,0.710000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,3285960.0,0.989071,2.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.065938,0.131876,0.000000,0.0,0.0,0.37000,0.680000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,3286073.0,0.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.77934,0.776331,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,3286113.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.77934,0.776331,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8,3286206.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.77934,0.776331,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [27]:
X= new_df.drop(df.columns[-1], axis = 1)
y = new_df['went_on_backorder']

In [28]:
X

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,3285085.0,0.999870,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016127,0.0,0.0,0.77934,0.776331,0.0,1.0,0.0,0.0,1.0,0.0
1,3285131.0,0.993884,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110432,0.0,0.0,0.77934,0.776331,0.0,0.0,0.0,1.0,0.0,0.0
2,3285358.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.92000,0.950000,0.0,0.0,0.0,0.0,1.0,0.0
3,3285517.0,0.976187,2.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.216930,0.000000,0.0,0.0,0.78000,0.750000,0.0,0.0,0.0,1.0,1.0,0.0
4,3285608.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.54000,0.710000,0.0,0.0,0.0,0.0,1.0,0.0
5,3285960.0,0.989071,2.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.065938,0.131876,0.000000,0.0,0.0,0.37000,0.680000,0.0,0.0,0.0,0.0,1.0,0.0
6,3286073.0,0.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.77934,0.776331,0.0,0.0,0.0,0.0,1.0,0.0
7,3286113.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.77934,0.776331,0.0,1.0,0.0,0.0,1.0,0.0
8,3286206.0,1.000000,8.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.77934,0.776331,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242076 entries, 0 to 242075
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sku                242076 non-null  object 
 1   national_inv       242075 non-null  float64
 2   lead_time          242076 non-null  float64
 3   in_transit_qty     242075 non-null  float64
 4   forecast_3_month   242075 non-null  float64
 5   forecast_6_month   242075 non-null  float64
 6   forecast_9_month   242075 non-null  float64
 7   sales_1_month      242075 non-null  float64
 8   sales_3_month      242075 non-null  float64
 9   sales_6_month      242075 non-null  float64
 10  sales_9_month      242075 non-null  float64
 11  min_bank           242075 non-null  float64
 12  potential_issue    242075 non-null  object 
 13  pieces_past_due    242075 non-null  float64
 14  perf_6_month_avg   242075 non-null  float64
 15  perf_12_month_avg  242075 non-null  float64
 16  lo

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [33]:
def transformation(df):
    # StandardScaler for scaling the values
    X= df.drop(df.columns[-1], axis = 1)
    #y = df['went_on_backorder'] 
    X_std = StandardScaler().fit_transform(X)

    # PCA for dimensionality reduction 
    dec = PCA(n_components=2)
    X_reduced = dec.fit_transform(X_std)
    return X_reduced
    


In [34]:
transformation(X)

array([[-2.4336325 ,  0.27803766],
       [-2.28954672,  0.81344668],
       [-2.03243846, -0.83484275],
       ...,
       [-2.66696866,  1.44781031],
       [-2.66104261,  1.44305739],
       [-2.54539605,  1.38518156]])

In [ ]:
X

KeyError: 'went_on_backorder'

In [17]:
df

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,3285085,62.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,No,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
1,3285131,9.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,No,0.0,-99.00,-99.00,0.0,No,No,Yes,No,No,No
2,3285358,17.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,0.92,0.95,0.0,No,No,No,Yes,No,No
3,3285517,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,No,0.0,0.78,0.75,0.0,No,No,Yes,Yes,No,No
4,3285608,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,0.54,0.71,0.0,No,No,No,Yes,No,No
5,3285960,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,No,0.0,0.37,0.68,0.0,No,No,No,Yes,No,No
6,3286073,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,-99.00,-99.00,0.0,No,No,No,Yes,No,No
7,3286113,28.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
8,3286206,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,-99.00,-99.00,0.0,Yes,No,Yes,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [18]:
y

0          No
1          No
2          No
3          No
4          No
5          No
6          No
7          No
8          No
9          No
10         No
11         No
         ... 
242064     No
242065     No
242066     No
242067     No
242068     No
242069     No
242070     No
242071     No
242072     No
242073     No
242074     No
242075    NaN
Name: went_on_backorder, Length: 242076, dtype: object